# Gemini Model Evaluation on Math and Science Questions

This notebook evaluates Google's Gemini model on 120 questions (60 math, 60 science) without hints.
The model receives ONLY the questions - ground truth answers are kept completely separate until evaluation.

## 1. Setup and Import Libraries

In [24]:
# Import required libraries
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
from typing import Dict, List, Tuple
import re
import google.generativeai as genai
from IPython.display import display, Markdown
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# Load environment variables from .env file
load_dotenv('../.env')

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully!")
print("Environment variables loaded from .env file")

Libraries imported successfully!
Environment variables loaded from .env file


## 2. Configure Gemini API

In [25]:
# Configure Gemini API
def setup_gemini():
    """Set up Gemini API with API key from .env file"""
    # Get API key from environment variable
    api_key = os.getenv('GEMINI_API_KEY')
    
    if not api_key:
        raise ValueError("GEMINI_API_KEY not found in .env file. Please add your API key to the .env file.")
    
    genai.configure(api_key=api_key)
    
    # Initialize the model - using gemini-2.0-flash (fast and has better free tier limits)
    model = genai.GenerativeModel('gemini-2.0-flash')
    print("✅ Gemini 2.0 Flash model initialized successfully using API key from .env!")
    print("   (Using Flash model for better free tier quota limits)")
    return model

# Initialize Gemini
model = setup_gemini()

✅ Gemini 2.0 Flash model initialized successfully using API key from .env!
   (Using Flash model for better free tier quota limits)


## 3. Load Questions and Separate Ground Truth

**IMPORTANT**: We load the questions and immediately separate them from ground truth answers.
The model will NEVER see the ground truth during inference.

In [26]:
def load_questions_and_separate_ground_truth(file_path: str) -> Tuple[List[Dict], Dict[int, str]]:
    """
    Load questions from JSON and separate ground truth.
    Returns:
        - questions_only: List of questions WITHOUT ground truth
        - ground_truth_dict: Separate dictionary of ground truth answers
    """
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    questions_only = []
    ground_truth_dict = {}
    
    for q in data['questions']:
        # Extract ONLY the information the model should see
        question_for_model = {
            'id': q['id'],
            'question': q['question'],
            'difficulty': q['difficulty'],
            'source': q.get('source', 'unknown')
        }
        
        # Add subject for science questions
        if 'subject' in q:
            question_for_model['subject'] = q['subject']
        
        questions_only.append(question_for_model)
        
        # Store ground truth separately
        ground_truth_dict[q['id']] = q['ground_truth']
    
    return questions_only, ground_truth_dict, data.get('metadata', {})

# Load math questions
math_questions, math_ground_truth, math_metadata = load_questions_and_separate_ground_truth(
    '../data/no-hints/math-questions.json'
)

# Load science questions  
science_questions, science_ground_truth, science_metadata = load_questions_and_separate_ground_truth(
    '../data/no-hints/science-questions.json'
)

print(f"📊 Loaded {len(math_questions)} math questions")
print(f"🔬 Loaded {len(science_questions)} science questions")
print(f"\n✅ Ground truth answers are stored separately and hidden from the model")
print(f"\nDifficulty distribution:")
print(f"  Math - Easy: {sum(1 for q in math_questions if q['difficulty'] == 'easy')}")
print(f"  Math - Medium: {sum(1 for q in math_questions if q['difficulty'] == 'medium')}")
print(f"  Math - Hard: {sum(1 for q in math_questions if q['difficulty'] == 'hard')}")
print(f"  Science - Easy: {sum(1 for q in science_questions if q['difficulty'] == 'easy')}")
print(f"  Science - Medium: {sum(1 for q in science_questions if q['difficulty'] == 'medium')}")
print(f"  Science - Hard: {sum(1 for q in science_questions if q['difficulty'] == 'hard')}")

📊 Loaded 60 math questions
🔬 Loaded 60 science questions

✅ Ground truth answers are stored separately and hidden from the model

Difficulty distribution:
  Math - Easy: 20
  Math - Medium: 20
  Math - Hard: 20
  Science - Easy: 20
  Science - Medium: 20
  Science - Hard: 20


## 4. Define Gemini Query Function (No Ground Truth Access)

In [27]:
def query_gemini(question_text: str, question_type: str = 'math') -> Dict:
    """
    Query Gemini with ONLY the question text.
    The model NEVER sees the ground truth answer.
    """
    if question_type == 'math':
        prompt = f"""Solve the following math problem. Provide ONLY the final numerical answer.
Do not include units or explanations, just the number.

Problem: {question_text}

Answer:"""
    else:  # science
        prompt = f"""Answer the following science question. Provide ONLY the answer.
Keep your answer concise and direct.

Question: {question_text}

Answer:"""
    
    try:
        start_time = time.time()
        response = model.generate_content(prompt)
        response_time = time.time() - start_time
        
        return {
            'answer': response.text.strip(),
            'response_time': response_time,
            'error': None
        }
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return {
            'answer': None,
            'response_time': None,
            'error': str(e)
        }

# Test with a sample question (WITHOUT showing ground truth)
sample_question = math_questions[0]
print(f"Sample Question ID {sample_question['id']}:")
print(f"Question: {sample_question['question']}")
print(f"
Querying Gemini...")
sample_response = query_gemini(sample_question['question'], 'math')

if sample_response['error']:
    print(f"❌ Error: {sample_response['error']}")
    print("
Please check your API key and internet connection.")
else:
    print(f"Gemini's Answer: {sample_response['answer']}")
    if sample_response['response_time']:
        print(f"Response Time: {sample_response['response_time']:.2f}s")
    print(f"
(Ground truth is hidden and will only be revealed during evaluation)")

SyntaxError: unterminated f-string literal (detected at line 43) (1501356254.py, line 43)

## 5. Process All Questions Through Gemini

Process all 120 questions. Gemini only sees the questions, NOT the answers.

In [23]:
def process_questions(questions: List[Dict], question_type: str) -> List[Dict]:
    """
    Process all questions through Gemini.
    Ground truth is NEVER passed to the model.
    """
    results = []
    total = len(questions)
    
    print(f"Processing {total} {question_type} questions...")
    print("="*50)
    
    for i, q in enumerate(questions, 1):
        # Query Gemini with ONLY the question text
        response = query_gemini(q['question'], question_type)
        
        result = {
            'id': q['id'],
            'question': q['question'],
            'difficulty': q['difficulty'],
            'gemini_answer': response['answer'],
            'response_time': response['response_time'],
            'error': response['error']
        }
        
        if 'subject' in q:
            result['subject'] = q['subject']
        
        results.append(result)
        
        # Progress update
        if i % 10 == 0 or i == total:
            print(f"Progress: {i}/{total} ({100*i/total:.1f}%)")
        
        # Rate limiting
        time.sleep(0.5)  # Avoid API rate limits
    
    print(f"✅ Completed processing {total} {question_type} questions\n")
    return results

# Process math questions
print("🔢 PROCESSING MATH QUESTIONS")
math_results = process_questions(math_questions, 'math')

# Process science questions
print("🔬 PROCESSING SCIENCE QUESTIONS")
science_results = process_questions(science_questions, 'science')

print("\n✅ All questions processed. Gemini has provided answers without seeing ground truth.")

🔢 PROCESSING MATH QUESTIONS
Processing 60 math questions...


KeyboardInterrupt: 

## 6. Evaluate Against Ground Truth

NOW we compare Gemini's answers with the ground truth that was kept hidden.

In [ ]:
def normalize_answer(answer: str, question_type: str = 'math') -> str:
    """
    Normalize answers for comparison.
    """
    if answer is None:
        return ""
    
    answer = str(answer).strip().lower()
    
    if question_type == 'math':
        # Extract numbers from the answer
        numbers = re.findall(r'-?\d+\.?\d*', answer)
        if numbers:
            # Convert to float then back to string to normalize
            try:
                num = float(numbers[0])
                # If it's a whole number, return as int
                if num.is_integer():
                    return str(int(num))
                return str(num)
            except:
                return numbers[0]
    else:  # science
        # Remove common words and punctuation for science answers
        answer = re.sub(r'[^a-z0-9\s]', '', answer)
        # Remove articles
        answer = re.sub(r'\b(the|a|an)\b', '', answer)
        answer = ' '.join(answer.split())  # Normalize whitespace
    
    return answer

def evaluate_results(results: List[Dict], ground_truth: Dict[int, str], question_type: str) -> pd.DataFrame:
    """
    Compare Gemini's answers with ground truth.
    This is the FIRST time we look at ground truth.
    """
    for result in results:
        q_id = result['id']
        
        # Get ground truth (first time accessing it)
        correct_answer = ground_truth[q_id]
        result['ground_truth'] = correct_answer
        
        # Normalize both answers for comparison
        normalized_gemini = normalize_answer(result['gemini_answer'], question_type)
        normalized_truth = normalize_answer(correct_answer, question_type)
        
        # Check if correct
        result['is_correct'] = normalized_gemini == normalized_truth
        result['normalized_gemini'] = normalized_gemini
        result['normalized_truth'] = normalized_truth
    
    return pd.DataFrame(results)

# Evaluate math results
math_df = evaluate_results(math_results, math_ground_truth, 'math')
print("📊 Math Results Evaluated")
print(f"Correct: {math_df['is_correct'].sum()}/{len(math_df)}")
print(f"Accuracy: {100 * math_df['is_correct'].mean():.2f}%\n")

# Evaluate science results
science_df = evaluate_results(science_results, science_ground_truth, 'science')
print("🔬 Science Results Evaluated")
print(f"Correct: {science_df['is_correct'].sum()}/{len(science_df)}")
print(f"Accuracy: {100 * science_df['is_correct'].mean():.2f}%\n")

# Combine all results
math_df['domain'] = 'math'
science_df['domain'] = 'science'
all_results_df = pd.concat([math_df, science_df], ignore_index=True)

print("📈 Overall Results")
print(f"Total Correct: {all_results_df['is_correct'].sum()}/{len(all_results_df)}")
print(f"Overall Accuracy: {100 * all_results_df['is_correct'].mean():.2f}%")

## 7. Detailed Analysis

In [ ]:
# Calculate accuracy metrics
def calculate_metrics(df: pd.DataFrame) -> Dict:
    """Calculate various accuracy metrics"""
    metrics = {}
    
    # Overall metrics
    metrics['overall_accuracy'] = df['is_correct'].mean() * 100
    metrics['total_correct'] = df['is_correct'].sum()
    metrics['total_questions'] = len(df)
    
    # By domain
    metrics['by_domain'] = df.groupby('domain')['is_correct'].agg(['mean', 'sum', 'count'])
    metrics['by_domain']['mean'] *= 100
    
    # By difficulty
    metrics['by_difficulty'] = df.groupby('difficulty')['is_correct'].agg(['mean', 'sum', 'count'])
    metrics['by_difficulty']['mean'] *= 100
    
    # By difficulty and domain
    metrics['by_difficulty_domain'] = df.groupby(['difficulty', 'domain'])['is_correct'].mean() * 100
    
    # For science, by subject
    science_df = df[df['domain'] == 'science']
    if 'subject' in science_df.columns:
        metrics['by_subject'] = science_df.groupby('subject')['is_correct'].agg(['mean', 'sum', 'count'])
        metrics['by_subject']['mean'] *= 100
    
    # Response time analysis
    metrics['avg_response_time'] = df['response_time'].mean()
    metrics['total_time'] = df['response_time'].sum()
    
    return metrics

metrics = calculate_metrics(all_results_df)

# Display comprehensive metrics
print("="*60)
print("COMPREHENSIVE PERFORMANCE METRICS")
print("="*60)

print(f"\n📊 OVERALL PERFORMANCE")
print(f"  Total Questions: {metrics['total_questions']}")
print(f"  Correct Answers: {metrics['total_correct']}")
print(f"  Accuracy: {metrics['overall_accuracy']:.2f}%")
print(f"  Average Response Time: {metrics['avg_response_time']:.2f}s")

print(f"\n📈 ACCURACY BY DOMAIN")
display(metrics['by_domain'])

print(f"\n📊 ACCURACY BY DIFFICULTY")
display(metrics['by_difficulty'])

print(f"\n🔬 SCIENCE ACCURACY BY SUBJECT")
if 'by_subject' in metrics:
    display(metrics['by_subject'])

print(f"\n📋 ACCURACY BY DIFFICULTY AND DOMAIN")
display(metrics['by_difficulty_domain'].unstack())

## 8. Visualizations

In [ ]:
# Create comprehensive visualizations
fig = plt.figure(figsize=(20, 15))

# 1. Overall Accuracy Comparison
ax1 = plt.subplot(3, 3, 1)
domains = ['Overall', 'Math', 'Science']
accuracies = [
    metrics['overall_accuracy'],
    metrics['by_domain'].loc['math', 'mean'],
    metrics['by_domain'].loc['science', 'mean']
]
colors = ['#3498db', '#e74c3c', '#2ecc71']
bars = ax1.bar(domains, accuracies, color=colors)
ax1.set_ylabel('Accuracy (%)')
ax1.set_title('Overall Accuracy Comparison')
ax1.set_ylim(0, 100)
for bar, acc in zip(bars, accuracies):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
             f'{acc:.1f}%', ha='center', va='bottom')

# 2. Accuracy by Difficulty
ax2 = plt.subplot(3, 3, 2)
difficulties = ['easy', 'medium', 'hard']
diff_acc = [metrics['by_difficulty'].loc[d, 'mean'] for d in difficulties]
bars = ax2.bar(difficulties, diff_acc, color=['#2ecc71', '#f39c12', '#e74c3c'])
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Accuracy by Difficulty Level')
ax2.set_ylim(0, 100)
for bar, acc in zip(bars, diff_acc):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
             f'{acc:.1f}%', ha='center', va='bottom')

# 3. Grouped Bar Chart - Difficulty by Domain
ax3 = plt.subplot(3, 3, 3)
x = np.arange(len(difficulties))
width = 0.35
math_accs = [metrics['by_difficulty_domain'].loc[(d, 'math')] for d in difficulties]
science_accs = [metrics['by_difficulty_domain'].loc[(d, 'science')] for d in difficulties]

ax3.bar(x - width/2, math_accs, width, label='Math', color='#e74c3c')
ax3.bar(x + width/2, science_accs, width, label='Science', color='#2ecc71')
ax3.set_xlabel('Difficulty')
ax3.set_ylabel('Accuracy (%)')
ax3.set_title('Accuracy by Difficulty and Domain')
ax3.set_xticks(x)
ax3.set_xticklabels(difficulties)
ax3.legend()
ax3.set_ylim(0, 100)

# 4. Pie Chart - Correct vs Incorrect
ax4 = plt.subplot(3, 3, 4)
sizes = [metrics['total_correct'], metrics['total_questions'] - metrics['total_correct']]
labels = [f'Correct\n({sizes[0]})', f'Incorrect\n({sizes[1]})']
colors = ['#2ecc71', '#e74c3c']
ax4.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
ax4.set_title('Overall Correct vs Incorrect')

# 5. Science Subject Performance
if 'by_subject' in metrics:
    ax5 = plt.subplot(3, 3, 5)
    subjects = metrics['by_subject'].index.tolist()
    subject_accs = metrics['by_subject']['mean'].tolist()
    bars = ax5.bar(subjects, subject_accs, color=['#9b59b6', '#3498db', '#e67e22'])
    ax5.set_ylabel('Accuracy (%)')
    ax5.set_title('Science Accuracy by Subject')
    ax5.set_ylim(0, 100)
    for bar, acc in zip(bars, subject_accs):
        ax5.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                 f'{acc:.1f}%', ha='center', va='bottom')

# 6. Response Time Distribution
ax6 = plt.subplot(3, 3, 6)
ax6.hist(all_results_df['response_time'].dropna(), bins=30, color='#3498db', alpha=0.7)
ax6.axvline(metrics['avg_response_time'], color='red', linestyle='--', 
            label=f'Mean: {metrics["avg_response_time"]:.2f}s')
ax6.set_xlabel('Response Time (seconds)')
ax6.set_ylabel('Frequency')
ax6.set_title('Response Time Distribution')
ax6.legend()

# 7. Heatmap - Performance Matrix
ax7 = plt.subplot(3, 3, 7)
heatmap_data = metrics['by_difficulty_domain'].unstack()
sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='RdYlGn', 
            vmin=0, vmax=100, ax=ax7, cbar_kws={'label': 'Accuracy (%)'})
ax7.set_title('Performance Heatmap: Difficulty × Domain')
ax7.set_xlabel('Domain')
ax7.set_ylabel('Difficulty')

# 8. Question-by-Question Performance (first 30 questions)
ax8 = plt.subplot(3, 3, 8)
sample_size = 30
sample_df = all_results_df.head(sample_size)
x_pos = np.arange(sample_size)
colors_map = {True: '#2ecc71', False: '#e74c3c'}
bar_colors = [colors_map[val] for val in sample_df['is_correct']]
ax8.bar(x_pos, [1]*sample_size, color=bar_colors, alpha=0.7)
ax8.set_xlabel('Question Number')
ax8.set_ylabel('Correct (Green) / Incorrect (Red)')
ax8.set_title(f'Performance on First {sample_size} Questions')
ax8.set_ylim(0, 1.2)
ax8.set_yticks([])

# 9. Cumulative Accuracy Over Time
ax9 = plt.subplot(3, 3, 9)
cumulative_correct = all_results_df['is_correct'].cumsum()
cumulative_total = np.arange(1, len(all_results_df) + 1)
cumulative_accuracy = (cumulative_correct / cumulative_total) * 100
ax9.plot(cumulative_accuracy, color='#3498db', linewidth=2)
ax9.axhline(y=metrics['overall_accuracy'], color='red', linestyle='--', 
            alpha=0.5, label=f'Final: {metrics["overall_accuracy"]:.1f}%')
ax9.set_xlabel('Question Number')
ax9.set_ylabel('Cumulative Accuracy (%)')
ax9.set_title('Cumulative Accuracy Over All Questions')
ax9.legend()
ax9.grid(True, alpha=0.3)

plt.suptitle('Gemini Model Performance Analysis', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## 9. Error Analysis

In [ ]:
# Analyze incorrect answers
incorrect_df = all_results_df[~all_results_df['is_correct']]

print("="*60)
print("ERROR ANALYSIS")
print("="*60)

print(f"
📊 Error Statistics:")
print(f"Total Errors: {len(incorrect_df)}")
print(f"Error Rate: {100 * len(incorrect_df) / len(all_results_df):.2f}%")

print(f"
📈 Errors by Domain:")
error_by_domain = incorrect_df['domain'].value_counts()
for domain, count in error_by_domain.items():
    total = len(all_results_df[all_results_df['domain'] == domain])
    print(f"  {domain.capitalize()}: {count}/{total} ({100*count/total:.1f}% error rate)")

print(f"
📊 Errors by Difficulty:")
error_by_difficulty = incorrect_df['difficulty'].value_counts()
for diff, count in error_by_difficulty.items():
    total = len(all_results_df[all_results_df['difficulty'] == diff])
    print(f"  {diff.capitalize()}: {count}/{total} ({100*count/total:.1f}% error rate)")

print(f"
🔬 Science Errors by Subject:")
science_errors = incorrect_df[incorrect_df['domain'] == 'science']
if 'subject' in science_errors.columns:
    error_by_subject = science_errors['subject'].value_counts()
    for subject, count in error_by_subject.items():
        total = len(science_df[science_df['subject'] == subject])
        print(f"  {subject.capitalize()}: {count}/{total} ({100*count/total:.1f}% error rate)")

# Show sample of errors
print("
" + "="*60)
print("SAMPLE OF INCORRECT ANSWERS (First 5)")
print("="*60)

for idx, row in incorrect_df.head(5).iterrows():
    print(f"
❌ Question {row['id']} ({row['domain']}, {row['difficulty']}):")
    question_text = row['question'][:100] + "..." if len(row['question']) > 100 else row['question']
    print(f"   Question: {question_text}")
    print(f"   Gemini Answer: {row['gemini_answer']}")
    print(f"   Correct Answer: {row['ground_truth']}")
    print(f"   Normalized Comparison: '{row['normalized_gemini']}' vs '{row['normalized_truth']}'")

## 10. Export Results

In [ ]:
# Save detailed results to CSV
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"gemini_evaluation_results_{timestamp}.csv"

# Prepare export dataframe
export_df = all_results_df[[
    'id', 'domain', 'difficulty', 'question', 
    'gemini_answer', 'ground_truth', 'is_correct',
    'response_time'
]].copy()

# Add subject for science questions
if 'subject' in all_results_df.columns:
    export_df['subject'] = all_results_df['subject']

# Save to CSV
export_df.to_csv(output_file, index=False)
print(f"✅ Results saved to: {output_file}")

# Create summary report
summary_report = f"""
GEMINI MODEL EVALUATION SUMMARY
================================
Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Model: Gemini 1.5 Pro

OVERALL RESULTS
---------------
Total Questions: {metrics['total_questions']}
Correct Answers: {metrics['total_correct']}
Overall Accuracy: {metrics['overall_accuracy']:.2f}%
Average Response Time: {metrics['avg_response_time']:.2f} seconds
Total Processing Time: {metrics['total_time']/60:.2f} minutes

PERFORMANCE BY DOMAIN
--------------------
Math Accuracy: {metrics['by_domain'].loc['math', 'mean']:.2f}% ({int(metrics['by_domain'].loc['math', 'sum'])}/{int(metrics['by_domain'].loc['math', 'count'])})
Science Accuracy: {metrics['by_domain'].loc['science', 'mean']:.2f}% ({int(metrics['by_domain'].loc['science', 'sum'])}/{int(metrics['by_domain'].loc['science', 'count'])})

PERFORMANCE BY DIFFICULTY
------------------------
Easy: {metrics['by_difficulty'].loc['easy', 'mean']:.2f}% ({int(metrics['by_difficulty'].loc['easy', 'sum'])}/{int(metrics['by_difficulty'].loc['easy', 'count'])})
Medium: {metrics['by_difficulty'].loc['medium', 'mean']:.2f}% ({int(metrics['by_difficulty'].loc['medium', 'sum'])}/{int(metrics['by_difficulty'].loc['medium', 'count'])})
Hard: {metrics['by_difficulty'].loc['hard', 'mean']:.2f}% ({int(metrics['by_difficulty'].loc['hard', 'sum'])}/{int(metrics['by_difficulty'].loc['hard', 'count'])})

KEY FINDINGS
-----------
• Best Performance: {('Math' if metrics['by_domain'].loc['math', 'mean'] > metrics['by_domain'].loc['science', 'mean'] else 'Science')} domain
• Most Challenging: {'Hard' if metrics['by_difficulty'].loc['hard', 'mean'] < metrics['by_difficulty'].loc['medium', 'mean'] and metrics['by_difficulty'].loc['hard', 'mean'] < metrics['by_difficulty'].loc['easy', 'mean'] else ('Medium' if metrics['by_difficulty'].loc['medium', 'mean'] < metrics['by_difficulty'].loc['easy', 'mean'] else 'Easy')} difficulty level
• Error Rate: {100 - metrics['overall_accuracy']:.2f}%

Note: Ground truth was completely isolated from the model during inference.
"""

# Save summary report
summary_file = f"gemini_evaluation_summary_{timestamp}.txt"
with open(summary_file, 'w') as f:
    f.write(summary_report)

print(f"\n📄 Summary report saved to: {summary_file}")
print("\n" + "="*60)
print(summary_report)

## 11. Final Summary and Insights

In [ ]:
# Generate final insights
display(Markdown("# 🎯 Final Evaluation Summary"))

display(Markdown(f"""
## Key Performance Indicators

| Metric | Value |
|--------|-------|
| **Overall Accuracy** | {metrics['overall_accuracy']:.2f}% |
| **Math Accuracy** | {metrics['by_domain'].loc['math', 'mean']:.2f}% |
| **Science Accuracy** | {metrics['by_domain'].loc['science', 'mean']:.2f}% |
| **Easy Questions** | {metrics['by_difficulty'].loc['easy', 'mean']:.2f}% |
| **Medium Questions** | {metrics['by_difficulty'].loc['medium', 'mean']:.2f}% |
| **Hard Questions** | {metrics['by_difficulty'].loc['hard', 'mean']:.2f}% |
| **Avg Response Time** | {metrics['avg_response_time']:.2f}s |

## 📊 Performance Analysis

### Strengths:
- Best performance on **{('Math' if metrics['by_domain'].loc['math', 'mean'] > metrics['by_domain'].loc['science', 'mean'] else 'Science')}** questions
- Highest accuracy on **{('Easy' if metrics['by_difficulty'].loc['easy', 'mean'] > metrics['by_difficulty'].loc['medium', 'mean'] and metrics['by_difficulty'].loc['easy', 'mean'] > metrics['by_difficulty'].loc['hard', 'mean'] else ('Medium' if metrics['by_difficulty'].loc['medium', 'mean'] > metrics['by_difficulty'].loc['hard', 'mean'] else 'Hard'))}** difficulty level
- Consistent response time averaging {metrics['avg_response_time']:.2f} seconds

### Areas for Improvement:
- Lower performance on **{('Math' if metrics['by_domain'].loc['math', 'mean'] < metrics['by_domain'].loc['science', 'mean'] else 'Science')}** questions
- Struggles with **{('Hard' if metrics['by_difficulty'].loc['hard', 'mean'] < metrics['by_difficulty'].loc['medium', 'mean'] and metrics['by_difficulty'].loc['hard', 'mean'] < metrics['by_difficulty'].loc['easy', 'mean'] else ('Medium' if metrics['by_difficulty'].loc['medium', 'mean'] < metrics['by_difficulty'].loc['easy', 'mean'] else 'Easy'))}** difficulty questions
- Overall error rate of {100 - metrics['overall_accuracy']:.2f}%

## 🔐 Evaluation Integrity

✅ **Ground Truth Isolation Verified**: 
- Questions were separated from answers before processing
- Gemini only received question text, never the correct answers
- Ground truth was only accessed during the evaluation phase
- This ensures unbiased model performance measurement
"""))

print("\n✅ Evaluation complete! All results have been saved.")